In [1]:
import pandas as pd
import numpy as np
import janitor
import pickle
import os
import operator
import functools
import itertools
from typing import Callable

import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
if 'raw_databases.pkl' not in os.listdir('./data/'):
    from quickstart.loader import XlsxDriveLoader

    Loads = XlsxDriveLoader()  # Drive Folder is hardcoded in module, since this is not prone to change
    databases = Loads.content  # Process takes approximately ~3 minutes to run for the first time.
                               # Then the file will be stored in the data/
else:
    with open('./data/raw_databases.pkl', 'rb') as file:
        databases = pickle.load(file)
    print("Loaded data from .pkl file")

Loaded data from .pkl file


In [3]:
(db_name_contaminantes, db_name_meteoorologicas, 
db_name_notaciones, db_name_estaciones) = databases.keys()


get_col_names: Callable[[pd.core.frame.DataFrame], pd.core.indexes.base.Index] = (
                         lambda df: sorted(list(df.columns)))

def shared_structure(db_name: str) -> list:
    """Returns sheets in dict of DataFrames that conain the same fields"""

    df_col_names = [[key, get_col_names(val)] for key, val in databases[db_name].items()]

    grouped = list()
    for _, g in itertools.groupby(df_col_names, operator.itemgetter(1)):
        group = list(g) 
        if len(group) > 1: # Can't be performed with comprehension since g is a generator/iterator
            grouped.append(group)
    
    if len(grouped) == 1: 
        return [sheet_name for sheet_name, _ in functools.reduce(operator.iconcat, grouped, [])] 
        # Expecting there is only one shared_structure across the DataFrames
    else:
        return ... # Not necessary

In [4]:
raw_contaminantes = pd.concat([databases[db_name_contaminantes][sheet] 
                            for sheet in shared_structure(db_name_contaminantes)])

raw_meteorologicas = pd.concat([databases[db_name_meteoorologicas][sheet] 
                            for sheet in shared_structure(db_name_meteoorologicas)])

# db_name_estaciones did not include information about our corresponding zone

In [5]:
fields = ['parametro', 'Fecha',
          'SO2', 'SO2 b'] # San Pedro identifiers

def munge(df: pd.core.frame.DataFrame, fields: list):
    return (
        df
        .loc[:, fields]
        .pipe(janitor.rename_columns, new_column_names={'parametro': 'molecule',
                                                        'Fecha': 'date'})
        .pipe(janitor.process_text, column_name='molecule', string_function='strip')
        .pivot(index='date', columns='molecule', values=['SO2', 'SO2 b'])
        .convert_dtypes()
        # Apply dictionary into df
        .pipe(janitor.clean_names, strip_underscores='r')
    )

In [6]:
contaminantes = munge(raw_contaminantes, fields)
meteorologicas = munge(raw_meteorologicas, fields)

In [7]:
# There is a Timestamp missing in df metereologicas
set(contaminantes.index).difference(set(meteorologicas.index))

{Timestamp('2019-05-15 20:00:00')}

In [8]:
df = contaminantes.join(other=meteorologicas,
                        how='outer')

In [9]:
df

so2                                       so2_b        \
molecule               co    no   no2   nox  o3 pm10 pm2_5  so2    co    no   
date                                                                          
2017-01-01 00:00:00   1.1  17.0  20.1  37.0   6  163  <NA>  5.0  <NA>  <NA>   
2017-01-01 01:00:00  1.05  19.5  25.1  44.5   5  117  <NA>  3.9  <NA>  <NA>   
2017-01-01 02:00:00  0.91  16.7  22.5  39.0   5   46  <NA>  3.5  <NA>  <NA>   
2017-01-01 03:00:00  0.75  12.5  16.8  29.2   8   36  <NA>  3.4  <NA>  <NA>   
2017-01-01 04:00:00  0.59   6.5   6.7  13.1  12   25  <NA>  3.2  <NA>  <NA>   
...                   ...   ...   ...   ...  ..  ...   ...  ...   ...   ...   
2021-06-30 19:00:00  0.67   4.4  11.3  15.7  17   36  14.0  3.5  <NA>  <NA>   
2021-06-30 20:00:00  0.66   3.9  10.7  14.6  16   41   7.0  3.6  <NA>  <NA>   
2021-06-30 21:00:00  0.64   3.4  11.7  15.1  15   16   7.0  3.4  <NA>  <NA>   
2021-06-30 22:00:00  0.61   3.4   9.7  13.1  16   31   6.0  3.1  <NA>  <NA>   
2021-06-30 23:00:00   0.6   3.5  12.0  15.5  14   37   6.0  3.3  <NA>  <NA>   

                     ...    so2             so2_b                          \
molecule             ...   tout  wdr    wsr   prs rainf    rh    sr  tout   
date                 ...                                                    
2017-01-01 00:00:00  ...  20.68  253    4.0  <NA>  <NA>  <NA>     l  <NA>   
2017-01-01 01:00:00  ...  21.59  244    5.4  <NA>  <NA>  <NA>     l  <NA>   
2017-01-01 02:00:00  ...  20.94  260    4.4  <NA>  <NA>  <NA>     l  <NA>   
2017-01-01 03:00:00  ...  20.99  257    3.7  <NA>  <NA>  <NA>     l  <NA>   
2017-01-01 04:00:00  ...  20.89   84    2.0  <NA>  <NA>  <NA>     l  <NA>   
...                  ...    ...  ...    ...   ...   ...   ...   ...   ...   
2021-06-30 19:00:00  ...  22.78   69   27.1  <NA>  <NA>  <NA>  <NA>  <NA>   
2021-06-30 20:00:00  ...  23.57   74   97.5  <NA>  <NA>  <NA>  <NA>  <NA>   
2021-06-30 21:00:00  ...  22.89   79  117.0  <NA>  <NA>  <NA>     l  <NA>   
2021-06-30 22:00:00  ...  22.72   89  117.7  <NA>  <NA>  <NA>     l  <NA>   
2021-06-30 23:00:00  ...  22.79   72  110.9  <NA>  <NA>  <NA>     l  <NA>   

                                 
molecule              wdr   wsr  
date                             
2017-01-01 00:00:00  <NA>  <NA>  
2017-01-01 01:00:00  <NA>  <NA>  
2017-01-01 02:00:00  <NA>  <NA>  
2017-01-01 03:00:00  <NA>  <NA>  
2017-01-01 04:00:00  <NA>  <NA>  
...                   ...   ...  
2021-06-30 19:00:00  <NA>  <NA>  
2021-06-30 20:00:00  <NA>  <NA>  
2021-06-30 21:00:00  <NA>  <NA>  
2021-06-30 22:00:00  <NA>  <NA>  
2021-06-30 23:00:00  <NA>  <NA>  

[39394 rows x 30 columns]

In [10]:
df.describe(include="all").T

count unique  top   freq        mean         std     min  \
      molecule                                                               
so2   co        29974.0    NaN  NaN    NaN    1.345601   57.773094 -9999.0   
      no        31724.0    NaN  NaN    NaN    8.802698   56.892955 -9999.0   
      no2       32615.0    NaN  NaN    NaN   10.720868   55.985114 -9999.0   
      nox       32627.0    NaN  NaN    NaN    19.42212   57.052338 -9999.0   
      o3        37937.0    NaN  NaN    NaN    25.12555   54.893801 -9999.0   
      pm10      36947.0    NaN  NaN    NaN    59.74929   63.465377 -9999.0   
      pm2_5     20461.0    NaN  NaN    NaN   17.459354  122.015373 -9999.0   
      so2       29468.0    NaN  NaN    NaN    3.866082   58.315631 -9999.0   
so2_b co           9445      6    a   9278         NaN         NaN     NaN   
      no           7693      6    l   4476         NaN         NaN     NaN   
      no2          6942      7    e   6643         NaN         NaN     NaN   
      nox          6787      6    l   3866         NaN         NaN     NaN   
      o3           1475      6    s    955         NaN         NaN     NaN   
      pm10         2473      8    o    933         NaN         NaN     NaN   
      pm2_5       18936      6    a  14349         NaN         NaN     NaN   
      so2          9943      6    x   5399         NaN         NaN     NaN   
so2   prs       38539.0    NaN  NaN    NaN  713.096253   54.592886 -9999.0   
      rainf     39030.0    NaN  NaN    NaN   -0.249726    50.61307 -9999.0   
      rh        38638.0    NaN  NaN    NaN   55.898908   74.836262 -9999.0   
      sr        39149.0    NaN  NaN    NaN   -0.079654   50.537001 -9999.0   
      tout      38594.0    NaN  NaN    NaN   21.977135   51.485418 -9999.0   
      wdr       38499.0    NaN  NaN    NaN  113.093015   82.178686 -9999.0   
      wsr       29533.0    NaN  NaN    NaN    4.529191   58.594841 -9999.0   
so2_b prs           742      4    x    324         NaN         NaN     NaN   
      rainf         416      3    x    201         NaN         NaN     NaN   
      rh            759      4    x    417         NaN         NaN     NaN   
      sr          18725      4    l  18437         NaN         NaN     NaN   
      tout          719      4    x    500         NaN         NaN     NaN   
      wdr           833      6    x    599         NaN         NaN     NaN   
      wsr          9743      5    s   8519         NaN         NaN     NaN   

                  25%    50%    75%    max  
      molecule                              
so2   co          0.8    1.5   2.39   5.79  
      no          3.2    8.2   11.0  161.0  
      no2         5.7    8.8   14.3   82.3  
      nox        12.3   16.9   23.3  198.2  
      o3         10.0   22.0   35.0  153.0  
      pm10       37.0   52.0   75.0  714.0  
      pm2_5       9.0   16.0   25.0  156.0  
      so2         2.9    3.6    4.8   67.4  
so2_b co          NaN    NaN    NaN    NaN  
      no          NaN    NaN    NaN    NaN  
      no2         NaN    NaN    NaN    NaN  
      nox         NaN    NaN    NaN    NaN  
      o3          NaN    NaN    NaN    NaN  
      pm10        NaN    NaN    NaN    NaN  
      pm2_5       NaN    NaN    NaN    NaN  
      so2         NaN    NaN    NaN    NaN  
so2   prs       712.3  713.2  714.3  738.9  
      rainf       0.0    0.0    0.0  17.99  
      rh         42.0   58.0   72.0   92.0  
      sr          0.0  0.004   0.31  0.971  
      tout      17.65   23.1  27.16  40.44  
      wdr        71.0   87.0  122.0  360.0  
      wsr         1.4    4.2    6.4  176.6  
so2_b prs         NaN    NaN    NaN    NaN  
      rainf       NaN    NaN    NaN    NaN  
      rh          NaN    NaN    NaN    NaN  
      sr          NaN    NaN    NaN    NaN  
      tout        NaN    NaN    NaN    NaN  
      wdr         NaN    NaN    NaN    NaN  
      wsr         NaN    NaN    NaN    NaN

Looks like only `so2_b` content has categorical variables.

Next:
- Replace $-9999.0$ values with `np.nan`
- Map categorical data with a dictionary
- One-hot encode
- Imputation, outlier analysis and standarize/normalize
- Begin with statistical analysis

In [11]:
with open('data/dataframe.pkl', 'wb') as file:
            pickle.dump(df, file)